In [87]:
import pandas as pd
# from sqlalchemy import create_engine

# Step 1: Load your CSV into a DataFrame
csv_path = r"C:\Users\User\Documents\GitHub\credit-risk-sentiment-nps\data\01_credit-risk-sentiment-nps-sample-orig.csv"
df1 = pd.read_csv(csv_path)
df1 = df.drop_duplicates(subset='CLIENTNUM')
df1.rename(columns={
    "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1": "NB_Attrition_Prob_Yes",
    "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2": "NB_Attrition_Prob_No"
}, inplace=True)

In [88]:
# Identify categorical columns
cat_cols = df1.select_dtypes(include='object').columns

# Display unique values
# for col in cat_cols:
#     print(f"\nUnique values in '{col}':")
#     print(df[col].unique())

# To set all 'Unknown' values in your categorical columns to NaN (null),
import numpy as np

# Step 1: Identify categorical columns
categorical_cols = df1.select_dtypes(include='object').columns

# Step 2: Replace 'Unknown' with np.nan in those columns
df1[categorical_cols] = df1[categorical_cols].replace('Unknown', np.nan)

# Identify categorical columns
cat_cols = df1.select_dtypes(include='object').columns

# Display unique values
# for col in cat_cols:
#     print(f"\nUnique values in '{col}':")
#     print(df[col].unique())

df = df1.dropna(subset=['Income_Category']).copy()

In [90]:
df.columns

Index(['CLIENTNUM', 'Attrition_Flag', 'Customer_Age', 'Gender',
       'Dependent_count', 'Education_Level', 'Marital_Status',
       'Income_Category', 'Card_Category', 'Months_on_book',
       'Total_Relationship_Count', 'Months_Inactive_12_mon',
       'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal',
       'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt',
       'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio',
       'NB_Attrition_Prob_Yes', 'NB_Attrition_Prob_No', 'Churn_Type',
       'annual_income', 'monthly_income', 'DTI_ratio', 'DTI_Risk_Level',
       'Credit_Score', 'Loan_Default_Flag', 'Number_of_Defaults',
       'Last_Default_Date', 'Default_Amount', 'Bankruptcy_History',
       'Bankruptcy_Flag', 'Adjusted_Credit_Score', 'Moodys_Rating',
       'Payment_History_Flag', 'Payment_Risk', 'estimated_annual_revenue'],
      dtype='object')

In [85]:
df.isnull().sum().sort_values(ascending=False)

Education_Level             1374
Marital_Status               667
CLIENTNUM                      0
Attrition_Flag                 0
Gender                         0
Customer_Age                   0
Dependent_count                0
Income_Category                0
Card_Category                  0
Months_on_book                 0
Total_Relationship_Count       0
Months_Inactive_12_mon         0
Contacts_Count_12_mon          0
Credit_Limit                   0
Total_Revolving_Bal            0
Avg_Open_To_Buy                0
Total_Amt_Chng_Q4_Q1           0
Total_Trans_Amt                0
Total_Trans_Ct                 0
Total_Ct_Chng_Q4_Q1            0
Avg_Utilization_Ratio          0
NB_Attrition_Prob_Yes          0
NB_Attrition_Prob_No           0
Churn_Type                     0
annual_income                  0
monthly_income                 0
DTI_ratio                      0
DTI_Risk_Level                 0
Credit_Score                   0
Loan_Default_Flag              0
Number_of_

In [72]:
import numpy as np
import pandas as pd

# Define income ranges for each category
income_ranges = {
    'Less than $40K': (20000, 39999),
    '$40K - $60K': (40000, 59999),
    '$60K - $80K': (60000, 79999),
    '$80K - $120K': (80000, 119999),
    '$120K +': (120000, 200000),  # or any reasonable upper bound
}

# Apply function to assign random incomes
def assign_random_income(row):
    category = row['Income_Category']
    if category in income_ranges:
        low, high = income_ranges[category]
        return np.random.randint(low, high + 1)
    else:
        return np.nan  # keep NaN for 'Unknown' or others

df['annual_income'] = df.apply(assign_random_income, axis=1)
df['monthly_income'] = df['annual_income'] /12
df['DTI_ratio'] = df['Total_Revolving_Bal']/df['monthly_income']*100

# to get Risk_Level
def classify_dti_risk(dti):
    if dti < 36:
        return 'Low Risk'
    elif 36 <= dti <= 49:
        return 'Medium Risk'
    else:
        return 'High Risk'

df['DTI_Risk_Level'] = df['DTI_ratio'].apply(classify_dti_risk)


In [73]:
# populate the 'Churn_Type' column  with alternating values "Voluntary" and "Involuntary" row by row:
import pandas as pd

# Get indices where Attrition_Flag is 'Attrited Customer'
churn_indices = df[df['Attrition_Flag'] == 'Attrited Customer'].index

alternating_values = ['Voluntary' if i % 4 == 1 else 'Involuntary' for i in range(len(churn_indices))]

# Create the Attrition_Type column with NaN first
df['Churn_Type'] = pd.NA

# Assign alternating values only to 'Attrited Customer' rows
df.loc[churn_indices, 'Churn_Type'] = alternating_values

df['Churn_Type'] = df['Churn_Type'].fillna('Existing Customer')



In [74]:
import numpy as np

# Set seed for reproducibility (optional)
np.random.seed(42)

# Assign random credit scores between 300 and 850 to 'Existing Customer' rows
df.loc[df['Attrition_Flag'] == 'Existing Customer', 'Credit_Score'] = np.random.randint(
    550, 851, size=(df['Attrition_Flag'] == 'Existing Customer').sum()
)

# Assign random credit scores between 300 and 850 to df['Churn_Type']'=='Voluntary'
df.loc[df['Churn_Type'] == 'Voluntary', 'Credit_Score'] = np.random.randint(
    550, 851, size=(df['Churn_Type'] == 'Voluntary').sum()
)

# Assign random credit scores between 300 and 850 to df['Churn_Type']'=='Involuntary'
df.loc[df['Churn_Type'] == 'Involuntary', 'Credit_Score'] = np.random.randint(
    450, 701, size=(df['Churn_Type'] == 'Involuntary').sum()
)

In [75]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

# Set random seed for reproducibility
np.random.seed(42)

# Populate  'Loan_Default_History' with binary flags: 1 = Defaulted, 0 = No Default
df['Loan_Default_Flag'] = np.where(
    df['Attrition_Flag'] == 'Attrited Customer',
    np.random.choice([1, 0], size=len(df), p=[0.2, 0.8]),
    np.random.choice([1, 0], size=len(df), p=[0.1, 0.9])
)

df['Loan_Default_Flag'] = np.where(
    df['Churn_Type'] == 'Involuntary',
    np.random.choice([1, 0], size=len(df), p=[0.4, 0.6]),
    np.random.choice([1, 0], size=len(df), p=[0.2, 0.8])
)

# Initialize empty columns
df['Number_of_Defaults'] = 0
df['Last_Default_Date'] = pd.NaT
df['Default_Amount'] = 0.0

# Get indices where customer defaulted
defaulted_idx = df[df['Loan_Default_Flag'] == 1].index

# Generate synthetic data for defaulted customers
df.loc[defaulted_idx, 'Number_of_Defaults'] = np.random.randint(1, 6, size=len(defaulted_idx))

# Generate random dates in the past 5 years
# def random_date_within_last_n_years(n):
#     days_ago = np.random.randint(30, 365 * n)
#     return datetime.today() - timedelta(days=int(days_ago))

# df.loc[defaulted_idx, 'Last_Default_Date'] = [random_date_within_last_n_years(5) for _ in range(len(defaulted_idx))]

df.loc[defaulted_idx, 'Last_Default_Date'] = [
    random_date_within_last_n_years(5) for _ in range(len(defaulted_idx))
]
df.loc[:, 'Last_Default_Date'] = df['Last_Default_Date'].fillna(pd.Timestamp('1900-01-01')) # example placeholder


# Generate random default amounts between $500 and $20,000
df.loc[defaulted_idx, 'Default_Amount'] = np.round(np.random.uniform(500, 20000, size=len(defaulted_idx)), 2)


In [76]:
import numpy as np

def assign_bankruptcy(row):
    if row['Attrition_Flag'] == 'Attrited Customer':
        if row['Churn_Type'] == 'Voluntary':
            return np.random.choice(['Yes', 'No'], p=[0.2, 0.8])
        elif row['Churn_Type'] == 'Involuntary':
            return np.random.choice(['Yes', 'No'], p=[0.4, 0.6])
        else:
            return 'No'
    elif row['Attrition_Flag'] == 'Existing Customer':
        return np.random.choice(['Yes', 'No'], p=[0.05, 0.95])
    else:
        return 'No'

df['Bankruptcy_History'] = df.apply(assign_bankruptcy, axis=1)


In [77]:
# Example data
import pandas as pd

# Create bankruptcy flag: 1 if history is 'Yes', else 0
df['Bankruptcy_Flag'] = df['Bankruptcy_History'].apply(lambda x: 1 if x.lower() == 'yes' else 0)

# print(df)

def adjust_score(row):
    if row['Bankruptcy_Flag'] == 1:
        # Example: deduct 150 points if bankruptcy exists
        return max(row['Credit_Score'] - 150, 300)  # Ensure minimum score 300
    else:
        return row['Credit_Score']

df['Adjusted_Credit_Score'] = df.apply(adjust_score, axis=1)

# print(df)

def credit_score_to_moodys(score):
    if score >= 800:
        return 'Aaa'
    elif score >= 750:
        return 'Aa'
    elif score >= 700:
        return 'A'
    elif score >= 650:
        return 'Baa'
    elif score >= 600:
        return 'Ba'
    elif score >= 550:
        return 'B'
    elif score >= 500:
        return 'Caa'
    elif score >= 400:
        return 'Ca'
    else:
        return 'C'

df['Moodys_Rating'] = df['Adjusted_Credit_Score'].apply(credit_score_to_moodys)

In [78]:
# : Payment history impact
import numpy as np

# Define a mask to filter only relevant rows (not bankrupt)
non_bankrupt_mask = ~df['Bankruptcy_History'].str.lower().eq('yes')

# Step 1: Start by filling with 'good' for all non-bankrupt customers
df.loc[non_bankrupt_mask, 'Payment_History_Flag'] = 'good'

# Rule 1: Bankruptcy => bad payment history
df.loc[df['Bankruptcy_History'].str.lower() == 'yes', 'Payment_History_Flag'] = 'bad'

# Rule 2: Involuntary attrition but not bankrupt => 35% bad, 65% good
mask_involuntary = (df['Churn_Type'] == 'Involuntary') & (df['Bankruptcy_History'].str.lower() != 'yes')
df.loc[mask_involuntary, 'Payment_History_Flag'] = np.where(
    np.random.rand(mask_involuntary.sum()) < 0.35, 'bad', 'good'
)

# Step 5: Payment history impact
df['Payment_Risk'] = df['Payment_History_Flag'].apply(lambda x: 'good' if x == 'good' else 'bad')
df['estimated_annual_revenue'] = df['Total_Revolving_Bal'] * 0.20 + 50


In [79]:

# Attrited Customer only
df_attr=  df[df['Attrition_Flag'] == 'Attrited Customer']
df_attr.to_csv( r"C:\Users\User\Documents\GitHub\credit-risk-sentiment-nps\data\02_churned_sample.csv",index = False)
df.to_csv( r"C:\Users\User\Documents\GitHub\credit-risk-sentiment-nps\data\03_processed_data_ready_to_analyze.csv",index = False)
